<a href="https://colab.research.google.com/github/Snigdha-Gayathri/Data-Mining/blob/main/CartoonDL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install tensorflow opencv-python matplotlib

In [5]:
!pip install tensorflow opencv-python matplotlib
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

# Function to load and preprocess an image
def load_and_process_img(path_to_img):
    img = tf.io.read_file(path_to_img)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = img[tf.newaxis, :]
    return img

# Function to deprocess an image for display
def deprocess_img(processed_img):
    x = processed_img.numpy()
    x = np.squeeze(x, 0)
    x = x * 255
    x = np.clip(x, 0, 255).astype('uint8')
    return x

# Load pre-trained VGG19 model
vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
vgg.trainable = False

# Define layers for content and style extraction
content_layers = ['block5_conv2']
style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']

# Function to extract content and style features
def vgg_layers(layer_names):
    outputs = [vgg.get_layer(name).output for name in layer_names]
    model = tf.keras.Model([vgg.input], outputs)
    return model

# Function to calculate content loss
def content_loss(target_content, generated_content):
    return tf.reduce_mean(tf.square(generated_content - target_content))

# Function to calculate gram matrix for style loss
def gram_matrix(input_tensor):
    result = tf.linalg.einsum('acdh,bcdh->abh', input_tensor, input_tensor)
    input_shape = tf.shape(input_tensor)
    num_locations = tf.cast(input_shape[1] * input_shape[2], tf.float32)
    return result / (num_locations)

# Function to calculate style loss
def style_loss(target_style, generated_style):
    return tf.reduce_mean(tf.square(generated_style - target_style))

# Function to calculate total loss
def total_loss(content_targets, style_targets, generated_outputs, content_weight, style_weight):
    content_outputs = generated_outputs[:1]
    style_outputs = generated_outputs[1:]

    content_loss_val = content_weight * content_loss(content_targets[0], content_outputs[0])

    style_loss_val = 0
    for target_style, generated_style in zip(style_targets, style_outputs):
        style_loss_val += style_loss(target_style, generated_style)
    style_loss_val *= style_weight / len(style_targets)

    total_loss_val = content_loss_val + style_loss_val
    return total_loss_val

# Function to perform style transfer
@tf.function()
def train_step(image, content_targets, style_targets, content_weight, style_weight, optimizer, extractor):
    with tf.GradientTape() as tape:
        outputs = extractor(image)
        loss = total_loss(content_targets, style_targets, outputs, content_weight, style_weight)
    grad = tape.gradient(loss, image)
    optimizer.apply_gradients([(grad, image)])
    image.assign(tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0))

def perform_style_transfer(content_path, style_path, output_path, epochs=10, steps_per_epoch=100, content_weight=1e3, style_weight=1e-2):
    content_image = load_and_process_img(content_path)
    style_image = load_and_process_img(style_path)

    extractor = vgg_layers(content_layers + style_layers)

    style_outputs = extractor(style_image)
    content_outputs = extractor(content_image)

    style_targets = [gram_matrix(output) for output in style_outputs[1:]]
    content_targets = content_outputs[:1]

    generated_image = tf.Variable(content_image)
    optimizer = tf.optimizers.Adam(learning_rate=0.02, beta_1=0.99, epsilon=1e-1)

    for n in range(epochs):
        for m in range(steps_per_epoch):
            train_step(generated_image, content_targets, style_targets, content_weight, style_weight, optimizer, extractor)
            print(".", end='', flush=True)
        print("Epoch {}".format(n))

    final_image = deprocess_img(generated_image)
    cv2.imwrite(output_path, cv2.cvtColor(final_image, cv2.COLOR_RGB2BGR))
    print(f"Style transfer complete. Image saved to {output_path}")

# Example usage:
content_image_path = '/content/GIRL.jpg'
style_image_path = '/content/HEIDI.webp'
output_image_path = '/content/styled_output.jpg'

perform_style_transfer(content_image_path, style_image_path, output_image_path)

img = cv2.imread(output_image_path)
if img is not None:
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.show()
else:
    print(f"Error: Could not read image from {output_image_path}")

InvalidArgumentError: {{function_node __wrapped__DecodeImage_device_/job:localhost/replica:0/task:0/device:CPU:0}} Unknown image file format. One of JPEG, PNG, GIF, BMP required. [Op:DecodeImage] name: 